In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

from models import VisualEncoder
from train import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder(features_dim)
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AtariCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.action_head = nn.Linear(features_dim, action_space.n)
        self.value_head = nn.Linear(features_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [3]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C17']))
    env = PovOnlyWrapper(env)
    env = IgluActionWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [5]:
from ray.tune.integration.wandb import WandbLogger

tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO C17 pretrained"
                  }
              }

        },
        loggers=[WandbLogger])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


Trial name,status,loc
PPO_my_env_82e40_00000,PENDING,


2021-09-18 10:02:20,060	INFO wandb.py:170 -- Already logged into W&B.
2021-09-18 10:02:20,070	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
(pid=35494) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=35494)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=35494) 2021-09-18 10:02:23,575	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=35494) 2021-09-18 10:02:23,575	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=35495) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=35495)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=35494) 2021-09-18 10:02:29,425	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=35495) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=35495)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=35495) /root/miniconda/envs/py37/lib/python3.7/

Result for PPO_my_env_82e40_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2021-09-18_10-03-29
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: -2.0
  episode_reward_mean: -2.0
  episode_reward_min: -2.0
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: 7b9a32d98e74411ea92d43be64b53a03
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999996
          cur_lr: 5.000000000000001e-05
          entropy: 1.3492794672648112
          entropy_coeff: 0.009999999999999998
          kl: 0.012402718118078582
          policy_loss: -0.025099800527095796
          total_loss: 0.01665419919623269
          vf_explained_var: 0.426866352558136
          vf_loss: 0.05276625651038355
    num_agent_steps_sampled: 1000
    num_agent_steps_trained: 1000
    num_steps_sampled: 1000
    num_steps_trained: 1000
  iterations_since_restore: 1
  node_ip: 192.168.3.5

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_82e40_00000,RUNNING,192.168.3.5:35494,142,1507.51,142000,-0.1,3,-7,996.03


(pid=35495) 2021-09-18 10:27:40,094	ERROR worker.py:428 -- SystemExit was raised from the worker
(pid=35495) Traceback (most recent call last):
(pid=35495)   File "python/ray/_raylet.pyx", line 640, in ray._raylet.task_execution_handler
(pid=35495)   File "python/ray/_raylet.pyx", line 488, in ray._raylet.execute_task
(pid=35495)   File "python/ray/_raylet.pyx", line 525, in ray._raylet.execute_task
(pid=35495)   File "python/ray/_raylet.pyx", line 532, in ray._raylet.execute_task
(pid=35495)   File "python/ray/_raylet.pyx", line 536, in ray._raylet.execute_task
(pid=35495)   File "python/ray/_raylet.pyx", line 486, in ray._raylet.execute_task.function_executor
(pid=35495)   File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/function_manager.py", line 563, in actor_method_executor
(pid=35495)     return method(__ray_actor, *args, **kwargs)
(pid=35495)   File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/util/iter.py", line 1151, in par_iter_next
(pid=

Exception ignored in: <function WandbLoggerCallback.__del__ at 0x7f4ec1a691e0>
Traceback (most recent call last):
  File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/tune/integration/wandb.py", line 378, in __del__
    for trial in self._trial_processes:
RuntimeError: dictionary changed size during iteration


agent_timesteps_total,142000
episode_len_mean,996.03
episode_reward_max,3.0
episode_reward_mean,-0.1
episode_reward_min,-7.0
episodes_this_iter,1
episodes_total,142
info/learner/default_policy/learner_stats/cur_kl_coeff,0.50625
info/learner/default_policy/learner_stats/cur_lr,5e-05
info/learner/default_policy/learner_stats/entropy,2.05332
info/learner/default_policy/learner_stats/entropy_coeff,0.01


agent_timesteps_total,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
episode_len_mean,████████▃▃▄▄▄▅▅▅▅▃▃▃▃▄▄▄▄▂▃▃▃▃▃▃▃▁▁▁▃▃▃▃
episode_reward_max,▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████████████████████
episode_reward_mean,▁▅▆▇▇▇▇█████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
episode_reward_min,████████████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episodes_this_iter,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episodes_total,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
info/learner/default_policy/learner_stats/cur_kl_coeff,▂▂▂▂▂▂▄▁▁▁▁▁▁▁▁▁▁▂▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅████
info/learner/default_policy/learner_stats/cur_lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
info/learner/default_policy/learner_stats/entropy,▁▄▆▅▆▆█▆▇▇▆▆▇▇▇▇▆▇▆▃▇▇█▅▆▆▇▇▆▆▅▆▆▃▄▄▄▃▆▅
info/learner/default_policy/learner_stats/entropy_coeff,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
